In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
config = {
    'host': 'http://localhost:9200',
    'source': {
        'index': 'records-hep',
        'analysis_field': 'abstracts.value',
        'date_field': 'earliest_date',
        'doc_type': 'hep',
    },
    'analysis': {
        'index': 'trends-analysis',
        'analysis_field': 'analysis',
        'date_field': 'date',
        'doc_type': 'hep',
    },
    'minimum_date': '2013-02-01',
    'maximum_date': None,
    'filter_script': "d = doc['earliest_date'].date; d.getDayOfYear() != 1",
    'unigram': True,
    'minimum_ngram': 2,
    'maximum_ngram': 3,
    'stopwords_file': '../invenio_trends/etl/stopwords.txt',
}

In [3]:
from invenio_trends.etl.index_synchronizer import IndexSynchronizer

In [4]:
etl = IndexSynchronizer(config)

etl.setup_index()
etl.setup_analyzer()
etl.setup_mappings()
etl.synchronize()

In [5]:
from invenio_trends.analysis.trends_detector import TrendsDetector

In [6]:
td = TrendsDetector(config)

In [7]:
date = '2015-01-31T00:00:00.00'
granularity = 'day'
foreground_window = 10
background_window = 365
minimum_frequency_threshold = 5
smoothing_len = 7
num_cluster = 3
num_trends = 1000

In [8]:
from invenio_trends.analysis.granularity import Granularity
from invenio_trends.analysis.utils import parse_iso_date

In [9]:
reference_date = parse_iso_date(date)
gran = Granularity[granularity]
foreground_start = reference_date - foreground_window * gran.value
background_start = reference_date - background_window * gran.value

ids = td.interval_ids(foreground_start, reference_date)
all_terms = td.term_vectors(ids)
terms = td.sorting_freq_threshold(all_terms, minimum_frequency_threshold)
hists = td.terms_histograms(terms, background_start, reference_date, gran)
scores = td.hist_scores(hists, foreground_start, smoothing_len)
trending = td.classify_scores(scores, num_cluster)
trends = td.prune_scores(trending, num_trends)

{'fields': ['none'], 'query': {'bool': {'filter': [{'exists': {'field': 'analysis'}}, {'range': {'date': {'lte': datetime.datetime(2015, 1, 31, 0, 0), 'gt': datetime.datetime(2015, 1, 21, 0, 0)}}}]}}}
[]


/usr/local/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required.

In [ ]:
td.interval_ids(foreground_start, reference_date)